In [1]:
from qiskit import *
import numpy as np
from scipy.stats import norm
from matplotlib import pyplot as plt
from scipy.stats import rv_continuous
from qiskit.circuit import Parameter, ParameterVector

In [2]:
def unitary(circ,eta,phi,t):
    theta = np.arccos(-eta);
    circ.u3(theta,phi,t,0);

In [3]:
class PQC:
    def __init__(self,name,layer):
        self.backend = Aer.get_backend('statevector_simulator');
        self.circ = QuantumCircuit(layer);
        self.name = name;
        self.seed = 14256;
        self.layer = layer;
        np.random.seed(self.seed);
        if self.name == "rcz":
            self.theta = Parameter('θ');
            for index in range(layer):
                self.circ.h(index);
            c = QuantumCircuit(1,name="Rz");
            c.rz(self.theta,0);
            temp = c.to_gate().control(1);
            self.circ.append(temp,[0,1]);
        if self.name == "circ4":
            self.theta1 = ParameterVector('θ1', layer);
            self.theta2 = ParameterVector('θ2', layer);
            self.theta3 = ParameterVector('θ3', layer-1);
            # print(self.theta1)
            for i in range(self.layer):
                self.circ.rx(self.theta1[i],i);
            for i in range(self.layer):
                self.circ.rz(self.theta2[i],i);
            for i in range(self.layer-1):
                c = QuantumCircuit(1, name="Rx");
                c.rz(self.theta3[i],0);
                temp = c.to_gate().control(1);
                self.circ.append(temp,[i,i+1]);

    def get(self):
        if self.name == "rcz":
            t = np.random.uniform(0,2*np.pi);
            # theta = Parameter('θ');
            self.circ1 = self.circ.assign_parameters({self.theta:t});
            result = execute(self.circ1,self.backend).result();
            out_state = result.get_statevector();
            self.statevector = np.asmatrix(out_state).T;
            return self.statevector;
        if self.name == "circ4":
            self.circ1 = self.circ.bind_parameters({self.theta1: np.random.uniform(0,2*np.pi,self.layer)});
            self.circ2 = self.circ1.bind_parameters({self.theta2: np.random.uniform(0,2*np.pi,self.layer)});
            self.circ3 = self.circ2.bind_parameters({self.theta3: np.random.uniform(0,2*np.pi,self.layer-1)});
            result = execute(self.circ3,self.backend).result();
            out_state = result.get_statevector();
            self.statevector = np.asmatrix(out_state).T;
            return self.statevector;

    def draw(self):
        self.circ.draw('mpl');
        print(self.circ);

In [4]:
pqc = PQC("circ4",5);

In [5]:
print(pqc.get())

[[-1.11501923e-01+4.84619369e-02j]
 [-8.71518095e-02+2.03410537e-01j]
 [-3.06029043e-03-4.43955186e-03j]
 [ 8.69259711e-03+4.55700296e-03j]
 [-4.15865303e-01-6.31064015e-02j]
 [-6.29543686e-01+4.35711242e-01j]
 [-8.08490347e-03-1.68122612e-02j]
 [ 2.72738094e-02+2.02273780e-02j]
 [ 3.44682665e-02+2.55718961e-02j]
 [ 7.75786806e-02-9.17435724e-03j]
 [-8.18966981e-04+1.71828750e-03j]
 [-1.37348527e-04-3.46195249e-03j]
 [-1.18455722e-01-8.95329657e-02j]
 [-2.68673208e-01+2.93426162e-02j]
 [ 3.18488178e-04-6.57778507e-03j]
 [ 5.06266441e-03+1.08652180e-02j]
 [ 2.24325343e-02-2.40846305e-02j]
 [-3.66016016e-04-5.99072256e-02j]
 [ 1.23958116e-03+7.70900960e-04j]
 [-2.65012691e-03-1.91004947e-04j]
 [ 1.10034848e-01-2.93073314e-02j]
 [ 1.09126998e-01-1.76211570e-01j]
 [ 3.82452095e-03+3.29826965e-03j]
 [-8.95618087e-03-2.07086654e-03j]
 [ 5.41756171e-03-1.02783936e-02j]
 [-5.65983404e-03-2.03768348e-02j]
 [ 4.93558133e-04+1.48118161e-04j]
 [-9.20557702e-04+1.79782376e-04j]
 [-1.90600504e-02+3.

In [6]:
pqc.draw()

     ┌───────────┐┌───────────┐                                       »
q_0: ┤ RX(θ1[0]) ├┤ RZ(θ2[0]) ├──────■────────────────────────────────»
     ├───────────┤├───────────┤┌─────┴─────┐                          »
q_1: ┤ RX(θ1[1]) ├┤ RZ(θ2[1]) ├┤ Rx(θ3[0]) ├──────■───────────────────»
     ├───────────┤├───────────┤└───────────┘┌─────┴─────┐             »
q_2: ┤ RX(θ1[2]) ├┤ RZ(θ2[2]) ├─────────────┤ Rx(θ3[1]) ├──────■──────»
     ├───────────┤├───────────┤             └───────────┘┌─────┴─────┐»
q_3: ┤ RX(θ1[3]) ├┤ RZ(θ2[3]) ├──────────────────────────┤ Rx(θ3[2]) ├»
     ├───────────┤├───────────┤                          └───────────┘»
q_4: ┤ RX(θ1[4]) ├┤ RZ(θ2[4]) ├───────────────────────────────────────»
     └───────────┘└───────────┘                                       »
«                  
«q_0: ─────────────
«                  
«q_1: ─────────────
«                  
«q_2: ─────────────
«                  
«q_3: ──────■──────
«     ┌─────┴─────┐
«q_4: ┤ Rx(θ3[3]) ├
«     └─

In [7]:
print(np.random.uniform(-1,1,5))

[ 0.827358   -0.77390047 -0.06319066  0.18777462  0.54397509]
